<a href="https://colab.research.google.com/github/giambono/divine_semantics/blob/main/notebooks/run_compute_qdrant_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Check if the notebook is running on Google Colab
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

# Clone the repository if running on Colab
if is_colab():
    print("Running on Google Colab. Cloning repository...")
    !git clone https://github.com/giambono/divine_semantics.git
    os.chdir("/content/divine_semantics")
    !pip install -r requirements.txt
else:
    import sys
    sys.path.append("..")
    print(f"Working directory set to: {os.getcwd()}")
    print("Not running on Google Colab.")

Running on Google Colab. Cloning repository...
Cloning into 'divine_semantics'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 92 (delta 33), reused 76 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 1.58 MiB | 7.81 MiB/s, done.
Resolving deltas: 100% (33/33), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4

In [2]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient

load_dotenv()

qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")

qdrant_client = QdrantClient(
    url=qdrant_url,
    api_key=qdrant_api_key,
)

In [6]:
import ast
import numpy as np
import pandas as pd
import src.compute_embeddings as compute_embeddings
from src.db_helper import get_db_connection
from src.utils import load_model


In [7]:

conn = get_db_connection()  # Ensure get_db_connection() is defined/imported
df = pd.read_sql_query("SELECT * FROM divine_comedy", conn)
verse_mappings = pd.read_sql_query("SELECT * FROM verse_mappings", conn)

# Merge the cumulative_indices column from verse_mappings into df
df = df.merge(
    verse_mappings[['cantica_id', 'canto', 'start_verse', 'end_verse', 'cumulative_indices']],
    on=['cantica_id', 'canto', 'start_verse', 'end_verse'],
    how='left'
)

# Convert the string to a list
df['cumulative_indices'] = df['cumulative_indices'].apply(
    lambda x: ast.literal_eval(x) if pd.notnull(x) else []
)


In [9]:
qdrant_client.get_collections()
#qdrant_client.delete_collection(collection_name="dante_multilingual_e5")

ResponseHandlingException: [Errno 99] Cannot assign requested address

In [ ]:
model_key = "multilingual_e5"
models = {model_key: load_model(model_key)}

compute_embeddings.compute_embeddings_and_upsert(df, models, qdrant_client, collection_name_prefix="dante_")

In [ ]:
collection = qdrant_client.get_collection("dante_multilingual_e5")

In [ ]:
all_points = []
offset = None  # Qdrant scroll API uses None as the initial offset

# Retrieve all points from the collection in batches
while True:
    scroll_result, next_offset = qdrant_client.scroll(
        collection_name="dante_multilingual_e5",
        limit=100,
        offset=offset,
        with_vectors=True,
        with_payload=True
    )
    if not scroll_result:
        break

    all_points.extend(scroll_result)
    offset = next_offset  # Use next_offset for pagination
    if next_offset is None:
        break

In [ ]:
import numpy as np
v = np.array(all_points[0].vector)
norm = np.linalg.norm(v)
norm